<a href="https://colab.research.google.com/github/StevenLevine-NOAA/NBM-Verif/blob/notebooks/Probs_and_Obs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [ ]:
#@title Initialize Notebook Part 2
#!mamba install -q -c conda-forge cartopy contextily pyproj pyepsg pygrib netCDF4
!pip install cartopy contextily pyproj pyepsg pygrib netCDF4
import numpy as np
from scipy.interpolate import CubicSpline as cs, UnivariateSpline as us
import pandas as pd
from urllib.request import urlretrieve, urlopen
import requests
from datetime import datetime, timedelta
import json
from netCDF4 import Dataset
import pygrib
import pyproj
from pyproj import Proj, transform
import os, re, traceback

import matplotlib
from matplotlib.colors import LinearSegmentedColormap
#from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.axes as maxes
import matplotlib.patheffects as PathEffects
from matplotlib.path import Path
from matplotlib.textpath import TextToPath
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
matplotlib.rcParams['font.sans-serif'] = 'Liberation Sans'
matplotlib.rcParams['font.family'] = "sans-serif"
from matplotlib.cm import get_cmap
import seaborn as sns

from cartopy import crs as ccrs, feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import contextily as cx
import itertools

import zipfile

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
background_color='#272727'
text_color='#121212'
map_land_color='#FAFAF8'
map_water_color='#D4DBDD'
map_border_color='grey'

rdate="202310251300"
vdate="202310271200"

obframe=pd.read_csv('allprobbs_'+rdate+'_'+vdate+'.csv',index_col=0)

nbm_init=datetime.strptime(rdate,'%Y%m%d%H%M')
valdate=datetime.strptime(vdate,'%Y%m%d%H%M')

matplotlib.rc('axes',facecolor=background_color,edgecolor=text_color)
fig_valid_date=valdate.strftime('%Y%m%d_%HZ')
valid_title=valdate.strftime('%HZ %a %m-%d-%Y')
nbm_init_title=nbm_init.strftime('%HZ %m-%d-%Y')
nbm_init_string=nbm_init.strftime('%Y%m%d_%H') + "Z"

print("Making plot for custom region!")
west = np.nanmin(obframe["lon"]) - 0.5
south = np.nanmin(obframe["lat"]) - 0.5
east = np.nanmax(obframe["lon"]) + 1.0
north = np.nanmax(obframe["lat"]) + 0.5
width, height = (16,9)
ratioxy = 16./9.
width_ratios = [ratioxy]#, 1]
proj = ccrs.PlateCarree()
print("Plot dimensions have been set up!")

fig = plt.figure(constrained_layout=True, figsize=(width,height), facecolor=background_color, frameon=True, dpi=150)
grid = fig.add_gridspec(1,1, hspace=0.1, width_ratios=width_ratios, height_ratios = [1], wspace=0.2)
ax1 = fig.add_subplot(grid[0,0], projection=ccrs.Mercator())
ax1.set_anchor('N')
ax1.set_facecolor(background_color)
ax1.set_extent([west, east, south, north], crs=proj)
ax1.add_feature(cfeature.OCEAN, edgecolor='none', facecolor=map_water_color, zorder=-2)
ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='none', facecolor=map_land_color, zorder=-1))
#ax1.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#272727', zorder=0)
ax1.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', edgecolor='none', facecolor=map_water_color, zorder=0))
ax1.add_feature(cfeature.BORDERS, edgecolor=map_border_color, facecolor='none', linewidth=2, zorder=1)
#ax1.add_feature(cfeature.NaturalEarthFeature('cultural', 'countries', '50m', edgecolor=map_border_color, facecolor='none', linewidth=2, zorder=2))
ax1.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor=map_border_color, facecolor='none', linewidth=1, zorder=2))

hitframe=obframe[obframe["Hit_1.0"] == "HIT"]
hitscat=ax1.scatter(hitframe["lon"].values,hitframe["lat"].values,color="limegreen",transform=proj)

FileNotFoundError: ignored